# torch.nn.ParameterDict(parameters=None)

- **ParameterDict** нь PyTorch-ийн параметрүүдийг түлхүүр-утга хослуулан хадгалах тусгай контейнер модуль юм. Энэ нь параметрүүдийг нэрээр нь хялбар удирдах, динамикаар нэмэх/хасах боломжийг олгодог.

In [1]:
import torch
import torch.nn as nn

class DynamicModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # ParameterDict үүсгэх
        self.params = nn.ParameterDict({
            'weight1': nn.Parameter(torch.randn(10, 5)),
            'bias1': nn.Parameter(torch.zeros(10)),
            'weight2': nn.Parameter(torch.randn(5, 2)),
            'bias2': nn.Parameter(torch.zeros(2))
        })
        
        # Хоосон ParameterDict эхлүүлж, дараа нь нэмэх
        self.extra_params = nn.ParameterDict()

In [2]:
model = DynamicModel()
model.extra_params['weight3'] = nn.Parameter(torch.randn(2, 1))
model.extra_params['bias3'] = nn.Parameter(torch.zeros(1))  
print(model)


DynamicModel(
  (params): ParameterDict(
      (bias1): Parameter containing: [torch.FloatTensor of size 10]
      (bias2): Parameter containing: [torch.FloatTensor of size 2]
      (weight1): Parameter containing: [torch.FloatTensor of size 10x5]
      (weight2): Parameter containing: [torch.FloatTensor of size 5x2]
  )
  (extra_params): ParameterDict(
      (weight3): Parameter containing: [torch.FloatTensor of size 2x1]
      (bias3): Parameter containing: [torch.FloatTensor of size 1]
  )
)


## Үндсэн үйлдлүүд:

### 1. Параметр нэмэх

In [3]:
# ParameterDict үүсгэх
param_dict = nn.ParameterDict()

# Нэг параметр нэмэх
param_dict['weight'] = nn.Parameter(torch.randn(3, 3))
param_dict['bias'] = nn.Parameter(torch.zeros(3))

# update() ашиглан олон параметр нэмэх
param_dict.update({
    'gamma': nn.Parameter(torch.ones(3)),
    'beta': nn.Parameter(torch.zeros(3))
})

### 2. Параметр авах, устгах

In [ ]:
# Түлхүүрээр параметр авах
weight_param = param_dict['weight']

# get() ашиглан авах (алга бол default утга буцаана)
bias_param = param_dict.get('bias', None)

# Параметр устгах
del param_dict['gamma']

# Түлхүүр байгаа эсэхийг шалгах
if 'beta' in param_dict:
    print("Beta parameter байна")

### 3. Удирдлага

In [4]:
# Бүх түлхүүрүүдийг авах
keys = param_dict.keys()
print(f"Түлхүүрүүд: {list(keys)}")

# Бүх утгуудыг авах
values = param_dict.values()

# Бүх параметрүүдийг давтах
for name, param in param_dict.items():
    print(f"{name}: {param.shape}")

# ParameterDict-ийн урт
print(f"Параметрийн тоо: {len(param_dict)}")

Түлхүүрүүд: ['weight', 'bias', 'beta', 'gamma']
weight: torch.Size([3, 3])
bias: torch.Size([3])
beta: torch.Size([3])
gamma: torch.Size([3])
Параметрийн тоо: 4


## Бодит хэрэглээний жишээ:

### 1. Динамик шингээлтийн давхарга

In [8]:
class AdaptiveLayer(nn.Module):
    def __init__(self, feature_sizes):
        super().__init__()
        self.feature_sizes = feature_sizes
        
        # Параметрүүдийг динамикаар үүсгэх
        self.params = nn.ParameterDict()
        
        for i, size in enumerate(feature_sizes):
            self.params[f'weight_{i}'] = nn.Parameter(torch.randn(size, size))
            self.params[f'bias_{i}'] = nn.Parameter(torch.zeros(size))
    
    def forward(self, x, feature_idx):
        # Тухайн шингээлтийн параметрүүдийг ашиглах
        weight = self.params[f'weight_{feature_idx}']
        bias = self.params[f'bias_{feature_idx}']
        
        return torch.matmul(x, weight) + bias

# Хэрэглээ
model = AdaptiveLayer([10, 20, 30])
print(model)
output = model(torch.randn(5, 10), feature_idx=0)
print(output.shape)  # гаралт: torch.Size([5, 10])

AdaptiveLayer(
  (params): ParameterDict(
      (weight_0): Parameter containing: [torch.FloatTensor of size 10x10]
      (bias_0): Parameter containing: [torch.FloatTensor of size 10]
      (weight_1): Parameter containing: [torch.FloatTensor of size 20x20]
      (bias_1): Parameter containing: [torch.FloatTensor of size 20]
      (weight_2): Parameter containing: [torch.FloatTensor of size 30x30]
      (bias_2): Parameter containing: [torch.FloatTensor of size 30]
  )
)
torch.Size([5, 10])


### 2. Олон оролттой сүлжээ

In [9]:
class MultiInputNetwork(nn.Module):
    def __init__(self, input_dims, hidden_dim, output_dim):
        super().__init__()
        
        # Оролт бүрийн тусдаа параметрүүд
        self.input_params = nn.ParameterDict()
        
        for i, dim in enumerate(input_dims):
            self.input_params[f'input_proj_{i}'] = nn.Parameter(
                torch.randn(dim, hidden_dim)
            )
        
        # Нийтлэг параметрүүд
        self.hidden_weight = nn.Parameter(torch.randn(hidden_dim, output_dim))
        self.hidden_bias = nn.Parameter(torch.zeros(output_dim))
    
    def forward(self, inputs):
        # inputs нь жагсаалт бөгөөд элемент бүр нь өөр оролтын өгөгдөл
        
        projections = []
        for i, x in enumerate(inputs):
            weight = self.input_params[f'input_proj_{i}']
            proj = torch.matmul(x, weight)
            projections.append(proj)
        
        # Бүх проекцийг нэгтгэх
        combined = torch.stack(projections).mean(dim=0)
        
        # Эцсийн давхарга
        output = torch.matmul(combined, self.hidden_weight) + self.hidden_bias
        
        return output

### 3. Хэсэгчилсэн шинэчлэлт

In [10]:
class SparseUpdateModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
        # Бүтэн embedding матриц
        self.full_embedding = nn.Parameter(torch.randn(vocab_size, embedding_dim))
        
        # Зөвхөн шинэчлэгдэх хэсгийн параметрүүд
        self.sparse_updates = nn.ParameterDict()
    
    def add_token_embeddings(self, token_ids, initial_values=None):
        """Шинэ токсинуудын embedding нэмэх"""
        for token_id in token_ids:
            if f'embed_{token_id}' not in self.sparse_updates:
                if initial_values is not None:
                    init_val = initial_values[token_id]
                else:
                    init_val = torch.randn(self.embedding_dim)
                
                self.sparse_updates[f'embed_{token_id}'] = nn.Parameter(init_val)
    
    def forward(self, token_ids):
        embeddings = []
        
        for token_id in token_ids:
            if f'embed_{token_id}' in self.sparse_updates:
                # Шинэчлэлт хийгдсэн embedding
                emb = self.sparse_updates[f'embed_{token_id}']
            else:
                # Анхны embedding
                emb = self.full_embedding[token_id]
            
            embeddings.append(emb)
        
        return torch.stack(embeddings)

### 4. Модулуудын параметрүүдийг бүлэглэх

In [11]:
class ModularNetwork(nn.Module):
    def __init__(self, num_modules, module_size):
        super().__init__()
        
        # Модуль бүрийн параметрүүдийг тусдаа хадгалах
        self.module_params = nn.ParameterDict()
        
        for i in range(num_modules):
            module_dict = nn.ParameterDict({
                f'W1_{i}': nn.Parameter(torch.randn(module_size, module_size)),
                f'b1_{i}': nn.Parameter(torch.zeros(module_size)),
                f'W2_{i}': nn.Parameter(torch.randn(module_size, module_size//2)),
                f'b2_{i}': nn.Parameter(torch.zeros(module_size//2))
            })
            
            self.module_params.update(module_dict)
    
    def get_module_parameters(self, module_id):
        """Тодорхой модулийн параметрүүдийг авах"""
        module_keys = [k for k in self.module_params.keys() if k.endswith(f'_{module_id}')]
        return {k: self.module_params[k] for k in module_keys}
    
    def forward(self, x, module_id):
        # Тухайн модулийн параметрүүдийг ашиглах
        W1 = self.module_params[f'W1_{module_id}']
        b1 = self.module_params[f'b1_{module_id}']
        W2 = self.module_params[f'W2_{module_id}']
        b2 = self.module_params[f'b2_{module_id}']
        
        h = torch.relu(torch.matmul(x, W1) + b1)
        output = torch.matmul(h, W2) + b2
        
        return output

## Дэвшилтэт хэрэглээ:
### 1. Хэрэглэгч тусгайлагдсан параметр

In [12]:
class PersonalizedModel(nn.Module):
    def __init__(self, global_dim, personal_dim):
        super().__init__()
        
        # Нийтлэг параметрүүд
        self.global_weight = nn.Parameter(torch.randn(global_dim, global_dim))
        
        # Хэрэглэгч тусгай параметрүүд
        self.personal_params = nn.ParameterDict()
    
    def add_user(self, user_id):
        """Шинэ хэрэглэгчийн параметр нэмэх"""
        if user_id not in self.personal_params.keys():
            # Хэрэглэгчийн шинэ параметр үүсгэх
            self.personal_params[f'user_{user_id}_weight'] = nn.Parameter(
                torch.randn(personal_dim, personal_dim) * 0.01  # Жижиг эхлэх утга
            )
            self.personal_params[f'user_{user_id}_bias'] = nn.Parameter(
                torch.zeros(personal_dim)
            )
    
    def forward(self, x, user_id=None):
        # Нийтлэг хувиргалт
        x = torch.matmul(x, self.global_weight)
        
        # Хэрэглэгч тусгай хувиргалт
        if user_id is not None:
            user_weight = self.personal_params[f'user_{user_id}_weight']
            user_bias = self.personal_params[f'user_{user_id}_bias']
            x = torch.matmul(x, user_weight) + user_bias
        
        return x

### 2. Таск тусгай параметр

In [14]:
class MultiTaskModel(nn.Module):
    def __init__(self, input_dim, task_dims):
        super().__init__()
        
        # Нийтлэг давхарга
        self.shared_layer = nn.Linear(input_dim, 64)
        
        # Таск тусгай параметрүүд
        self.task_heads = nn.ParameterDict()
        
        for task_name, output_dim in task_dims.items():
            self.task_heads[f'{task_name}_weight'] = nn.Parameter(
                torch.randn(64, output_dim)
            )
            self.task_heads[f'{task_name}_bias'] = nn.Parameter(
                torch.zeros(output_dim)
            )
    
    def forward(self, x, task_name):
        # Нийтлэг шинж чанарууд
        features = torch.relu(self.shared_layer(x))
        
        # Таск тусгай толгой
        weight = self.task_heads[f'{task_name}_weight']
        bias = self.task_heads[f'{task_name}_bias']
        
        output = torch.matmul(features, weight) + bias
        
        return output

# Хэрэглээ
tasks = {'classification': 10, 'regression': 1, 'segmentation': 20}
model = MultiTaskModel(100, tasks)

# Өөр өөр таск дээр ашиглах
x = torch.randn(5, 100)
cls_output = model(x, 'classification')
reg_output = model(x, 'regression')

## Чухал шинж чанарууд:
### ParameterDict vs Python dict

In [15]:
# Буруу арга
class WrongModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.params = {}  # Энгийн dict
        self.params['weight'] = nn.Parameter(torch.randn(3, 3))
        # Параметр бүртгэгдэхгүй!

# Зөв арга
class CorrectModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.params = nn.ParameterDict()  # ParameterDict
        self.params['weight'] = nn.Parameter(torch.randn(3, 3))
        # Параметр автоматаар бүртгэгдэнэ!

### Параметрийн удирдлага

In [20]:
import torch
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(f"Using device: {device}")

Using device: mps


In [21]:
model = nn.ParameterDict({
    'weight': nn.Parameter(torch.randn(5, 3)),
    'bias': nn.Parameter(torch.zeros(3))
})

# Бүх параметрүүдийг харах
print("Бүх параметрүүд:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.shape}")

# Градиент идэвхжүүлэх/унтраах
model['weight'].requires_grad = False

# Параметр дамжуулах
model.to(device)  # GPU эсвэл CPU (MPS)руу шилжүүлэх
model.float()     # Float төрөлд хүртэлх

Бүх параметрүүд:
  bias: torch.Size([3])
  weight: torch.Size([5, 3])


ParameterDict(
    (bias): Parameter containing: [torch.mps.FloatTensor of size 3]
    (weight): Parameter containing: [torch.mps.FloatTensor of size 5x3]
)

## Практик зөвлөмж:

- **Нэршил:** Тогтвортой нэршил ашиглах (жишээ нь: 'layer1_weight', 'layer1_bias')

- **Динамик параметр:** Тохиргооноос хамаарч параметр нэмэх/хасах

- **Сонгон суралцах:** Зөвхөн зарим параметрүүдийг сургах

- **Сериалчлал:** ParameterDict нь state_dict-д зөв хадгалагдана

- **Солих боломж:** Модулуудын хооронд параметр солилцох

## Давуу талууд:

- **Уян хатан:** Динамикаар параметр нэмэх/хасах

- **Цэвэр код:** Параметрийн удирдлагыг хялбаршуулдаг

- **Бүрэн дүүрэн:** Бүх PyTorch параметрийн давуу талууд

- **Хялбар сервисчлэл:** state_dict ашиглан хялбар хадгалах/дуурайх

ParameterDict нь нарийн төвөгтэй, динамик нейрон сүлжээний загваруудад параметрүүдийг зохион байгуулахад маш ашигтай хэрэгсэл юм.